In [40]:
import geopandas as gpd
import pandas as pd
from PIL import Image, ImageFilter
import pillow_heif
from pillow_heif import HeifImagePlugin
from PIL.ExifTags import TAGS
import glob
from GPSPhoto import gpsphoto
import os
from datetime import datetime

In [41]:

def get_image_info(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    for tag, value in exif_data.items():
        if TAGS.get(tag) == 'DateTime':
            date_time_obj = datetime.strptime(value, "%Y:%m:%d %H:%M:%S")
        
        gps_info = gpsphoto.getGPSData(image_path)
        classification = os.path.split(image_path)[0]
        classification_info = classification.split("\\")[1][3:]

    abs_path = os.path.abspath(image_path)

    return abs_path, classification_info, date_time_obj, gps_info['Latitude'], gps_info['Longitude'], gps_info['Altitude']

In [43]:
road_use_images = glob.glob("./01 Road Use Classification/*/*.HEIC")

df = pd.DataFrame(columns=["Image", "Classification", "DateTime", "Latitude", "Longitude", "Altitude"])


for image_path in road_use_images:
    abs_path, classification_info, date_time_obj, gps_info_latitude, gps_info_longitude, gps_info_altitude = get_image_info(image_path)
    new_entry = pd.DataFrame(
        {
            "Image": abs_path,
            "Classification": [classification_info],
            "DateTime": [date_time_obj],
            "Latitude": [gps_info_latitude],
            "Longitude": [gps_info_longitude],
            "Altitude": [gps_info_altitude]
        }
    )
    df = pd.concat([df, new_entry], ignore_index=True)


df

C:\Users\HP\AppData\Local\Temp\ipykernel_15480\3171263854.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_entry], ignore_index=True)


,Image,Classification,DateTime,Latitude,Longitude,Altitude
0,c:\Users\HP\Documents\PhilSA\road-classificati...,Motorcycle Taxis,2023-06-05 05:01:58,10.769764,106.700369,13.892330
1,c:\Users\HP\Documents\PhilSA\road-classificati...,Motorcycle Taxis,2023-06-05 05:02:13,10.769831,106.700417,6.608486
2,c:\Users\HP\Documents\PhilSA\road-classificati...,Motorcycle Taxis,2023-06-05 05:03:57,10.770211,106.700797,10.646498
3,c:\Users\HP\Documents\PhilSA\road-classificati...,Motorcycle Taxis,2023-06-05 05:06:02,10.770400,106.700936,2.427700
4,c:\Users\HP\Documents\PhilSA\road-classificati...,Motorcycle Taxis,2023-06-05 05:06:26,10.770411,106.700989,0.872250
...,...,...,...,...,...,...
1215,c:\Users\HP\Documents\PhilSA\road-classificati...,Others,2023-06-05 21:43:14,10.769011,106.700189,11.653461
1216,c:\Users\HP\Documents\PhilSA\road-classificati...,Others,2023-06-05 21:43:24,10.769069,106.700142,11.640495
1217,c:\Users\HP\Documents\PhilSA\road-classificati...,Others,2023-06-05 21:43:42,10.769206,106.700028,12.283968
1218,c:\Users\HP\Documents\PhilSA\road-classificati...,Others,2023-06-05 21:43:56,10.769283,106.699975,12.362715


In [44]:
df.to_csv("roadclassification.csv")